# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [1]:
# Do all imports and installs here
# Loading all library
import numpy as np
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window
import boto3
import time
from functools import reduce
import pandas as pd
# Change padans parameter to adjust visliazation
pd.set_option('max_colwidth', 200)
pd.set_option('display.max_columns', 200)
# Load in aws credential
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))
KEY = config.get('AWS','KEY')
SECRET = config.get('AWS','SECRET')
os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['KEY']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['SECRET']

In [2]:
# Define a function that creates spark session
start_time = time.time()
def create_spark_session():
    """
    This function is used to create a spark session to work in
    """
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark
# Create spark session
spark = create_spark_session()

In [3]:
# Load data from AWS s3
#df_spark = spark.read.csv("/home/workspace/ibrd-statement-of-loans-historical-data.csv", header=True)

df_spark = spark.read.csv("s3a://udacity-leejohn/loan/ibrd-statement-of-loans-historical-data.csv", header=True)
# function that will uppercase everything in the dataframe
fields = df_spark.schema.fields
stringFields = filter(lambda f: isinstance(f.dataType, StringType), fields)
nonStringFields = map(lambda f: col(f.name), filter(lambda f: not isinstance(f.dataType, StringType), fields))
stringFieldsTransformed = map(lambda f: upper(col(f.name)), stringFields) 
allFields = [*stringFieldsTransformed, *nonStringFields]
df_new = df_spark.select(allFields)
# rename the column name
# Get old column names 
oldColumns = df_new.schema.names
# Setup new column names
newColumns  = ['End_of_Period', 'Loan_Number', 'Region', 'Country_Code', 'Country','Borrower','Guarantor_Country_Code','Guarantor','Loan_Type','Loan_Status','Interest_Rate','Currency_of_Commitment','Project_ID','Project_Name','Original_Principal_Amount','Cancelled_Amount','Undisbursed_Amount','Disbursed_Amount','Repaid_to_IBRD','Due_to_IBRD','Exchange_Adjustment','Borrowers_Obligation','Sold_3rd_Party','Repaid_3rd_Party','Due_3rd_Party','Loans_Held','First_Repayment_Date','Last_Repayment_Date','Agreement_Signing_Date','Board_Approval_Date','Effective_Date_Most_Recent','Closed_Date_Most_Recent','Last_Disbursement_Date']
# Rename the dataframe
df = reduce(lambda df_spark, idx: df_spark.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), df_new)

In [4]:
# This is to display the number of null value of each column, before we do any cleaning, you will see null data count for each column with below command
df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns)).toPandas()

,End_of_Period,Loan_Number,Region,Country_Code,Country,Borrower,Guarantor_Country_Code,Guarantor,Loan_Type,Loan_Status,Interest_Rate,Currency_of_Commitment,Project_ID,Project_Name,Original_Principal_Amount,Cancelled_Amount,Undisbursed_Amount,Disbursed_Amount,Repaid_to_IBRD,Due_to_IBRD,Exchange_Adjustment,Borrowers_Obligation,Sold_3rd_Party,Repaid_3rd_Party,Due_3rd_Party,Loans_Held,First_Repayment_Date,Last_Repayment_Date,Agreement_Signing_Date,Board_Approval_Date,Effective_Date_Most_Recent,Closed_Date_Most_Recent,Last_Disbursement_Date
0,0,0,0,0,0,5557,31542,59135,48,48,26382,802089,31389,198899,48,48,48,48,48,48,48,48,48,48,48,48,3822,3726,10075,96,4699,842,351895


In [5]:
# Since below 4 columns have too much null values(average 50% null values) and are not necessary, so we drop them
df = df.drop('Last_Disbursement_Date')
df = df.drop('Currency_of_Commitment')
df = df.drop('Borrower')
df = df.drop('Project_Name')

In [6]:
# edit the loan_number to make it 9 digits
# Find records with 9 digits Loan_Number, which is the correct data format we are looking for
df_good = df.filter(length(col("Loan_Number")) == 9)

# Find records that have less than 9 digits Loan_Number, replace them to the correct format
df_6 = df.where(length(col("Loan_Number")) == 6).withColumn("Loan_Number", regexp_replace(col("Loan_Number") ,  "(\\w{4})(\\d{2})" , "$1000$2" ))
df_7 = df.where(length(col("Loan_Number")) == 7).withColumn("Loan_Number", regexp_replace(col("Loan_Number") ,  "(\\w{4})(\\d{3})" , "$100$2" ))
df_8 = df.where(length(col("Loan_Number")) == 8).withColumn("Loan_Number", regexp_replace(col("Loan_Number") ,  "(\\w{4})(\\d{4})" , "$10$2" ))

# No records with loan_number that has less than 9 digits
df = df_good.union(df_6).union(df_7).union(df_8)

In [7]:
# if both Guarantor_Country_Code and Guarantor are empty, then it's hard to say whether they are suppose to be empty(no guarantor)
# or they are missing values, so I just drop them.
df = df.filter('Guarantor_Country_Code is not NULL or Guarantor is not NULL')

In [8]:
# For each loan number, there should be one country code. Run below code, we will find there are 3 records that 
#df.select('Loan_Number','Country_Code').distinct().groupBy('Loan_Number').count().withColumnRenamed('count', 'ccount').filter('count>1').toPandas()
#withColumnRenamed('count', 'ccount').groupBy('Loan_Number').count().filter('count>1').toPandas()
#xx = df.select('Loan_Number','Country').distinct().groupBy('Loan_Number').count().filter('count>1').select('Loan_Number').collect()
#xarr = [str(xx[i].Loan_Number) for i in range(len(xx))]
#for i in xarr:
#    print(i)
#df.where(df.Loan_Number == 'IBRD82610').select('Country').groupBy('Country').count().toPandas()
#df.where(df.Loan_Number == 'IBRD82550').select('Country').groupBy('Country').count().toPandas()
#df.where(df.Loan_Number == 'IBRD82580').select('Country').groupBy('Country').count().toPandas()
df = df.where((df.Loan_Number == 'IBRD82580') & (df.Country != 'CHINA') | (df.Loan_Number != 'IBRD82580'))
df = df.where((df.Loan_Number == 'IBRD82550') & (df.Country != 'CHINA') | (df.Loan_Number != 'IBRD82550'))
df = df.where((df.Loan_Number == 'IBRD82610') & (df.Country != 'INDIA') | (df.Loan_Number != 'IBRD82610'))
# For each loan number, there should be one country code. Run below code, we will find there are 3 records that 
#df.select('Loan_Number','Country_Code').distinct().groupBy('Loan_Number').count().withColumnRenamed('count', 'ccount').filter('count>1').toPandas()
#withColumnRenamed('count', 'ccount').groupBy('Loan_Number').count().filter('count>1').toPandas()

In [9]:
# Remove all missing value for Project_ID column
# Create a dict that key is loan_number and value is project_id
x1 = df.filter('Project_ID is not NULL').select('Loan_Number','Project_ID').distinct().toPandas().set_index('Loan_Number')['Project_ID'].to_dict() 
# Create a pandas dataframe that only has the records that have missing project_id
pdf = df.filter('Project_ID is NULL').toPandas()
# Loop through the dataframe and replace its missing value with the loan_number
for index,row in pdf.iterrows():
    att = row.Loan_Number
    if att in x1.keys():
        row.Project_ID = x1.get(att)
# Convert it back to a spark dataframe
ddd = spark.createDataFrame(pdf.astype(str)).filter('Project_ID is not NULL')
# Union it with the good records and get new dataframe
df = df.filter('Project_ID is not NULL').union(ddd)

In [10]:
# Remove all missing value for Guarantor and Guarantor_Country_Code column
# Create a dict that key is loan_number and value is a array of country, country_code
c1 = df.select('Loan_Number','Country','Country_Code').distinct().toPandas().set_index('Loan_Number').T.to_dict('list')
# there is one more country in Guarantor, which is united kingdom, we will add it to our dict
c1b = df.filter('Guarantor == \'UNITED KINGDOM\'').select('Loan_Number').distinct().collect()
for i in c1b:
    c1[i.Loan_Number] = ['UNITED KINGDOM','GB']
# Loop through the dataframe and replace its missing value with the loan_number
pdf = df.filter('Guarantor is NULL or Guarantor_Country_Code is NULL').toPandas()
for index,row in pdf.iterrows():
    att = row.Loan_Number
    if att in c1.keys():
        row.Guarantor = c1.get(att)[0]
        row.Guarantor_Country_Code = c1.get(att)[1]
# Convert it back to a spark dataframe
ddd = spark.createDataFrame(pdf).filter('Guarantor is not NULL and Guarantor_Country_Code is not NULL')
# Union it with the good records and get new dataframe
df = df.filter('Guarantor is not NULL and Guarantor_Country_Code is not NULL').union(ddd)

In [11]:
# Remove all missing value for Interest_Rate
# Since for each loan number, we will find different interest rate. So to handle this case, for rach loan number, I will use mean value of all intertest rate to repalce its nul value. 
# Create a dict that key is loan_number and value is intertest rate
i1 = df.filter('Interest_Rate is not NULL').filter('Interest_Rate != \'None\'').select('Loan_Number','Interest_Rate').distinct().toPandas()
i1.Interest_Rate = i1.Interest_Rate.astype(np.float16)
i1 = i1.groupby('Loan_Number', as_index=True).agg({"Interest_Rate": "mean"})['Interest_Rate'].to_dict()
pdf = df.filter('Interest_Rate is NULL').toPandas()
# Loop through the dataframe and replace its missing value with the loan_number
for index,row in pdf.iterrows():
    att = row.Interest_Rate
    if att in c1.keys():
        row.Interest_Rate = c1.get(att)
# Convert it back to a spark dataframe
ddd = spark.createDataFrame(pdf.astype(str)).filter('Interest_Rate is not NULL')
# Union it with the good records and get new dataframe
df = df.filter('Interest_Rate is not NULL').union(ddd)

In [12]:
# Build a function that will take a dataframe and a column and replace its null value with the same method from above cases.
def clean(df, column):
    x1 = df.filter('{} is not NULL'.format(column)).filter('{} != \'None\''.format(column)).select('Loan_Number','{}'.format(column)).distinct().toPandas().set_index('Loan_Number')['{}'.format(column)].to_dict()
    pdf = df.filter('{} is NULL'.format(column)).toPandas()
    for index,row in pdf.iterrows():
        att = row.Loan_Number
        if att in x1.keys():
            row.column = x1.get(att)
    ddd = spark.createDataFrame(pdf.astype(str)).filter('{} is not NULL'.format(column)).filter('{} != \'None\''.format(column))
    df = df.filter('{} is not NULL'.format(column)).union(ddd)
    return df

In [13]:
# use this function to repalce all missing values for the date related column.
df = clean(df,'First_Repayment_Date')
df = clean(df,'Agreement_Signing_Date')
df = clean(df,'Closed_Date_Most_Recent')
df = clean(df,'Effective_Date_Most_Recent')
df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns)).toPandas()
print("--- %s seconds ---" % (time.time() - start_time))

--- 1087.240961074829 seconds ---


For all above we clean our data and now there's no null value in the data anymore. this process will take about 17 minutes.


Start from here, we will create file for fact table and dimension table

In [14]:
# Generate country dataframe 
df_country = df.select('Country_Code','Country','Region').distinct()
newRow = spark.createDataFrame([('GB','United Kingdom','EUROPE AND CENTRAL ASIA')])
df_country = df_country.union(newRow)
# add a unique id Country_Id for this dataframe
df_country = df_country.withColumn('Country_Id',row_number().over(Window.orderBy(monotonically_increasing_id())))
#df_country.limit(2).toPandas()

In [15]:
# Generate time dataframe 
# By exploring data, we can see that for each loan number, it should only have one set of dates related column. However, we will find for some column, it has more than one combination. so i crete this function to find its more frequent value for each loan number. 
def getss(df, column):
    # create a sql table with loan number and column
    df.select('Loan_Number', column).createOrReplaceTempView("time")
    # run query to get the most frequent value of each category for each loan number
    x = spark.sql("""
    SELECT Loan_Number, {} FROM 
    (
    SELECT Loan_Number, {}, count(1) total_records, ROW_NUMBER() OVER (PARTITION BY Loan_Number ORDER BY count(1) desc) AS seqnum
    FROM time 
    group by Loan_Number, {}
    )
    WHERE seqnum = 1
    """.format(column, column, column))
    return x
x1 = getss(df, 'First_Repayment_Date')
x2 = getss(df, 'Last_Repayment_Date')
x3 = getss(df, 'Agreement_Signing_Date')
x4 = getss(df, 'Board_Approval_Date')
x5 = getss(df, 'Effective_Date_Most_Recent')
x6 = getss(df, 'Closed_Date_Most_Recent')
df_time = x1.join(x2, x1.Loan_Number == x2.Loan_Number).drop(x2.Loan_Number)
df_time = df_time.join(x3, df_time.Loan_Number == x3.Loan_Number).drop(x3.Loan_Number)
df_time = df_time.join(x4, df_time.Loan_Number == x4.Loan_Number).drop(x4.Loan_Number)
df_time = df_time.join(x5, df_time.Loan_Number == x5.Loan_Number).drop(x5.Loan_Number)
df_time = df_time.join(x6, df_time.Loan_Number == x6.Loan_Number).drop(x6.Loan_Number)
df_time = df_time.withColumn('Time_Id',row_number().over(Window.orderBy(monotonically_increasing_id())))
df_time = df_time.selectExpr('Time_Id', 'Loan_Number', 'First_Repayment_Date as First_Repayment_Date_t', 'Last_Repayment_Date as Last_Repayment_Date_t', 'Agreement_Signing_Date as Agreement_Signing_Date_t','Board_Approval_Date \
                         as Board_Approval_Date_t','Effective_Date_Most_Recent as Effective_Date_Most_Recent_t','Closed_Date_Most_Recent as Closed_Date_Most_Recent_t')

In [16]:
# Generate Amount dataframe 
df_amount = df.select('Loan_Number','Original_Principal_Amount','Sold_3rd_Party','Repaid_3rd_Party', 'Due_3rd_Party').distinct()
# add a unique id Amount_Id for this dataframe
df_amount = df_amount.withColumn('Amount_Id',row_number().over(Window.orderBy(monotonically_increasing_id())))

In [17]:
# Generate Loan_Type dataframe 
df_loan_type = df.select('Loan_Type').distinct()
# add a unique id Loan_Type_Id for this dataframe
df_loan_type = df_loan_type.withColumn('Loan_Type_Id',row_number().over(Window.orderBy(monotonically_increasing_id())))

In [18]:
# Generate Loan_Status dataframe 
df_loan_status = df.select('Loan_Status').distinct()
# add a unique id Loan_Status_Id for this dataframe
df_loan_status = df_loan_status.withColumn('Loan_Status_Id',row_number().over(Window.orderBy(monotonically_increasing_id())))

In [19]:
# create sql tables 
df_country.createOrReplaceTempView("country")
df_time.createOrReplaceTempView("time")
df_amount.createOrReplaceTempView("amount")
df_loan_type.createOrReplaceTempView("loan_type")
df_loan_status.createOrReplaceTempView("loan_status")
df.createOrReplaceTempView("log")

In [20]:
# Generate transaction dataframe by joing log table with the other tables on loan number
log_norm = spark.sql("""
    select l.Loan_Number, t.Time_Id, c.Country_Id, cc.Country_Id as Guarantor_Country_Id, lt.Loan_Type, ls.Loan_Status_Id, a.Amount_Id,
    l.End_of_Period, l.Interest_Rate, l.Project_ID, l.Exchange_Adjustment, l.Borrowers_Obligation, l.Cancelled_Amount,
    l.Undisbursed_Amount, l.Disbursed_Amount, l.Repaid_to_IBRD, l.Due_to_IBRD, l.Loans_Held
    from log l 
    left join country c on l.Country_Code= c.Country_Code
    left join country cc on l.Guarantor = cc.Country 
    left join loan_type lt on l.Loan_Type = lt.Loan_Type 
    left join loan_status ls on l.Loan_Status = ls.Loan_Status
    left join amount a on l.Loan_Number = a.Loan_Number
    left join time t on l.Loan_Number = t.Loan_Number
    """)

In [21]:
# drop the loan number column for the dataframes
df_amount_final = df_amount.select('Amount_Id','Original_Principal_Amount','Sold_3rd_Party','Repaid_3rd_Party', 'Due_3rd_Party')
df_time_final = df_time.selectExpr('Time_Id', 'First_Repayment_Date_t as First_Repayment_Date', 'Last_Repayment_Date_t as Last_Repayment_Date',\
                                   'Agreement_Signing_Date_t as Agreement_Signing_Date','Board_Approval_Date_t \
                                     as Board_Approval_Date','Effective_Date_Most_Recent_t as Effective_Date_Most_Recent','Closed_Date_Most_Recent_t as Closed_Date_Most_Recent')

Saving files to S3, this process will take about 20 minutes.

In [ ]:
#start_time = time.time()
#df_country.write.csv("/home/workspace/csv/country.csv")
#df_time_final.write.parquet("s3a://udacity-leejohn/loan/time.parquet")
#df_loan_status.write.parquet("s3a://udacity-leejohn/loan/loan_status.parquet")
#df_loan_type.write.parquet("s3a://udacity-leejohn/loan/loan_type.parquet")
#df_amount_final.write.parquet("s3a://udacity-leejohn/loan/amount.parquet")
#log_norm.write.parquet("s3a://udacity-leejohn/loan/log_nor.parquet")
#print("--- %s seconds ---" % (time.time() - start_time))

In [24]:
start_time = time.time()
df_country.write.csv("/home/workspace/csv/country.csv")
df_time_final.write.csv("/home/workspace/csv/time.csv")
df_loan_status.write.csv("/home/workspace/csv/loan_status.csv")
df_loan_type.write.csv("/home/workspace/csv/loan_type.csv")
df_amount_final.write.csv("/home/workspace/csv/amount.csv")
log_norm.write.csv("/home/workspace/csv/log_nor.csv")
print("--- %s seconds ---" % (time.time() - start_time))

--- 53.77740550041199 seconds ---


Run below script will create a redshift cluster and it will take about 5-10 min

In [ ]:
%run -i 'Redshift.py'

In [ ]:
# Read in the data here
s3 = boto3.resource('s3',
                       region_name="us-east-1",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )
## This is to display what we have in S3 
sampleDbBucket =  s3.Bucket("john-udacity-s3")
i = 0
for obj in sampleDbBucket.objects.filter(Prefix="movie"):
    print(obj)
    i += 1
    if i > 10:
        break  

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

In [6]:
os.getcwd()

'/home/workspace'

In [23]:
# This is used to delete output file
def Delete_All(file_path):
    folder = file_path
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e)
Delete_All('/home/workspace/csv')
#Delete_All('data/log_data')
#Delete_All('Output/')

name 'shutil' is not defined
name 'shutil' is not defined
name 'shutil' is not defined
name 'shutil' is not defined
name 'shutil' is not defined
name 'shutil' is not defined
name 'shutil' is not defined
